# Домашнее задание 2. Извлечение коллокаций + NER

Выберите корпус отзывов на товары одной из категорий Amazon:
http://jmcauley.ucsd.edu/data/amazon/

Допустим, что вам нужно подготовить аналитический отчет по этим отзывам — например, для производителя нового продукта этой категории. Для этого будем искать упоминания товаров в отзывах (будем считать их NE). Учтите, что упоминание может выглядеть не только как "Iphone 10", но и как "модель", "телефон" и т.п.

**Важное замечание**: в задании приводятся примеры решений, вы можете их использовать!

1. (3 балла) Предложите 3 способа найти упоминания товаров в отзывах. 
Например, использовать bootstrapping: составить шаблоны вида "холодильник XXX", найти все соответствующие n-граммы и выделить из них называние товара.
Могут помочь заголовки и дополнительные данные с Amazon (Metadata [здесь](https://nijianmo.github.io/amazon/index.html))
Какие данные необходимы для каждого из способов? Какие есть достоинства/недостатки?

1) Bootstrapping. Посмотреть корпус, вручную выделить контексты, в которых фигурирует название товара. Далее по этим шаблонам выделить все сущности.

Необходимые данные. 
- Корпус 
- Придуманные шаблоны

Достоинства. 
+ Можем вручную контролировать результат: добавлять или убирать часть шаблонов

Недостатки. 
- Формат отзывов - свободный, поэтому потенциальных шаблонов может быть бесконечно много, а выделять их придётся долго
- Название товара может вообще не упоминаться в отзыве и мы не сможем его выделить
- Смешиваются дескрипторы и NE - название товара
- Мы заранее не знаем длину NE, но нам надо её задать в шаблоне. Если мы будем искать дескриптор + униграмму, то потеряем часть многих названий. Если будем искать дескриптор + длинную n-грамму, то наберём лишних слов

2) Вручную составить список возможных названий товаров. Найти для них синонимы с помощью эмбеддинга, используя косинусную близость

Необходимые данные.
- Корпус/метаданные для составления первоначального списка
- Эмбеддинг для поиска синонимов

Достоинства.
+ Как и в прошлом пункте, можем добавлять/убирать нужные названия в изначальный список по своему желанию

Недостатки.
- Данные эмбеддинга могут значительно отличаться от данных корпуса
- Нужно вручную подбирать порог косинусной близости
- Нужно придумывать первончальный список названий. Не факт, что наше воображение подскажет наиболее центральные наименования, от которых хорошо найдутся остальные (синонимы)

3) Извлечь из метаданных название товара - существительное. По корпусу отзывов найти для них синонимы, используя косинусную близость.

Необходимые данные. 
- Корпус 
- Метаданные
- Векторная модель для поиска синонимов

Достоинства.
+ Извлекаем данные автоматически
+ Все данные актуальны, так как берём их из самого корпуса, а не из внешних моделей или словарей

Недостатки.
- Необходимо подобрать порог косинусной близости
- Не извлечем упоминания, далёкие от названия товара

2. (2 балла) Реализуйте один из предложенных вами способов.

Примеры в качестве подсказки (можно использовать один из них): 
- написать правила с помощью [natasha/yargy](https://github.com/natasha/yargy)
- составить мини-словарь сущностей/дескрипторов, расширить с помощью эмбеддингов (например, word2vec)

Категория: Музыкальные инструменты

Попробую реализовать способ 3

In [48]:
import json
import gzip
from tqdm import tqdm
import random
from collections import Counter
from gensim.models import Word2Vec
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
sw = stopwords.words('english')
sc = spacy.load("en_core_web_sm")

In [2]:
# загружаем данные 

def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield json.loads(l)
        

data = tuple(tqdm(parse('reviews_Musical_Instruments_5.json.gz')))

10261it [00:00, 22360.35it/s]


In [3]:
len(data)

10261

In [4]:
data[-11]

{'reviewerID': 'A30J7WQV0ZNRXG',
 'asin': 'B00JBIVXGC',
 'reviewerName': 'D. Reinstein "marindavid"',
 'helpful': [0, 0],
 'reviewText': 'I had used Elixer strings for several years on both 6 and 12 string guitars. Somehow, I had gravitated over to Martin PB about a year ago (when I acquired a new Martin... Surprise!) I got a set of these strings through the Vine program and chose to try them out on a Gibson Songwriter Studio Deluxe CE RATHER than on a Taylor or Martin. The result is outstanding - So much so, that I am beginning to restring all 7 of my acoustic and acoustic-electric guitars with Elixers again.They are even smoother and easier to play that I remembered them. Of course, these new strings have improved the technology which becomes more and more important to me as my fingers age right along with the rest of my body. The sound is sweet and warm and the Gibson has never sounded better - played plugged-in or unplugged.Elixer definitely still has the edge. Thanks to Bob Taylor

In [5]:
# загружаем метаданные
meta = tuple(tqdm(parse('meta_Musical_Instruments.json.gz')))

120310it [01:06, 1797.65it/s]


In [6]:
len(meta)

120310

In [7]:
meta[0]

{'category': ['Musical Instruments',
  'Drums & Percussion',
  'Hand Percussion',
  'Folk & World',
  'Latin Percussion',
  'Guiros'],
 'tech1': '',
 'description': ["Cricket Rubbing the spine with the wooden stick up and down motion on its back making cricket chirping like buzzing or sizzling sound. Owl Whistle its back producing lovely high-low sounds. It would produce Owl's sound like 00-oo- hoo-hoo-hoo-hoo-oo-00. These products are 100% natural and handmade by local Thai artisan causing variations in its design and color. * Choose your owl design, either all brown or with black foot. Default is all brown."],
 'fit': '',
 'title': 'Wooden Percussion 2 Piece Set of 3 Inch Cricket and Inch Owl',
 'also_buy': ['B00NP8GYVS', 'B00NP80XMO', 'B00NP8M098'],
 'tech2': '',
 'brand': 'WADSUWAN SHOP',
 'feature': ['Wood percussion',
  'Owl whistle*',
  'Includes wooden scraper',
  'Small 3"',
  'Age rating: 3+'],
 'rank': ['>#141,729 in Musical Instruments (See Top 100 in Musical Instruments)',

In [10]:
# tqdm показал, что полный набор метаданных будет обрабатываться час. чтобы не тратить время, сделаем рандомную выборку
sample = random.sample(meta, 10000)

nouns = []

for item in tqdm(sample):
    title = item['title']
    parsed = sc(title)
    for token in parsed:
        if token.pos_ == 'NOUN':
            nouns.append(token.lemma_)

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [05:03<00:00, 32.98it/s]


In [12]:
cnt = Counter(nouns)

In [13]:
cnt.most_common(100)

[('inch', 443),
 ('amp', 186),
 ('case', 159),
 ('pack', 143),
 ('usb', 130),
 ('microphone', 128),
 ('foot', 118),
 ('cable', 92),
 ('size', 83),
 ('string', 83),
 ('ft', 81),
 ('x', 73),
 ('|', 66),
 ('dj', 60),
 ('set', 60),
 ('g', 58),
 ('mm', 58),
 ('style', 50),
 ('light', 46),
 ('piece', 44),
 ('way', 44),
 ('leather', 39),
 ('guitar', 39),
 ('strap', 39),
 ('planet', 38),
 ('c', 38),
 ('pc', 37),
 ('gold', 36),
 ('performance', 36),
 ('cd', 36),
 ('line', 32),
 ('hole', 32),
 ('medium', 31),
 ('saxophone', 30),
 ('end', 29),
 ('sound', 29),
 ('accessory', 29),
 ('pair', 27),
 ('pin', 26),
 ('effect', 26),
 ('stand', 25),
 ('instrument', 25),
 ('fender', 24),
 ('stage', 24),
 ('cm', 23),
 ('dvd', 23),
 ('plug', 23),
 ('system', 22),
 ('color', 22),
 ('boss', 21),
 ('silver', 21),
 ('pick', 21),
 ('a', 21),
 ('audio', 21),
 ('abs', 21),
 ('pro', 21),
 ('sd', 20),
 ('wheel', 20),
 ('unit', 20),
 ('volume', 20),
 ('tuning', 20),
 ('clip', 20),
 ('o', 19),
 ('type', 19),
 ('machine'

Мы получили некоторое количество мусора. Попробуем отфильтровать по длине слова. Плюс удалим inch - это, очевидно, часть характеристики товара, а не его название

In [15]:
del cnt['inch']

In [19]:
to_del = []

for word in cnt:
    if len(word) <= 2:
        to_del.append(word)
        
for word in to_del:
    del cnt[word]

In [100]:
cnt.most_common(100)

[('amp', 186),
 ('case', 159),
 ('pack', 143),
 ('usb', 130),
 ('microphone', 128),
 ('foot', 118),
 ('cable', 92),
 ('size', 83),
 ('string', 83),
 ('set', 60),
 ('style', 50),
 ('light', 46),
 ('piece', 44),
 ('way', 44),
 ('leather', 39),
 ('guitar', 39),
 ('strap', 39),
 ('planet', 38),
 ('gold', 36),
 ('performance', 36),
 ('line', 32),
 ('hole', 32),
 ('medium', 31),
 ('saxophone', 30),
 ('end', 29),
 ('sound', 29),
 ('accessory', 29),
 ('pair', 27),
 ('pin', 26),
 ('effect', 26),
 ('stand', 25),
 ('instrument', 25),
 ('fender', 24),
 ('stage', 24),
 ('dvd', 23),
 ('plug', 23),
 ('system', 22),
 ('color', 22),
 ('boss', 21),
 ('silver', 21),
 ('pick', 21),
 ('audio', 21),
 ('abs', 21),
 ('pro', 21),
 ('wheel', 20),
 ('unit', 20),
 ('volume', 20),
 ('tuning', 20),
 ('clip', 20),
 ('type', 19),
 ('machine', 19),
 ('pedal', 19),
 ('headset', 19),
 ('drumstick', 18),
 ('year', 17),
 ('flute', 17),
 ('mixer', 17),
 ('side', 17),
 ('ovation', 16),
 ('key', 16),
 ('channel', 16),
 ('led

In [101]:
# Так уже лучше

chosen = set([item[0] for item in cnt.most_common(100)])

In [112]:
corpus = []

for item in tqdm(data):
    text = item['reviewText']
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalpha() and word not in sw]
    corpus.append(words)

100%|███████████████████████████████████████████████████████████████████████████| 10261/10261 [00:25<00:00, 406.14it/s]


In [113]:
corpus[0]

['not',
 'much',
 'write',
 'exactly',
 'supposed',
 'filters',
 'pop',
 'sounds',
 'recordings',
 'much',
 'crisp',
 'one',
 'lowest',
 'prices',
 'pop',
 'filters',
 'amazon',
 'might',
 'well',
 'buy',
 'honestly',
 'work',
 'despite',
 'pricing']

In [114]:
model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=1, workers=4)

In [115]:
model.wv.most_similar('guitar', topn=10)

[('electric', 0.9032512903213501),
 ('acoustic', 0.8921219110488892),
 ('guitars', 0.8815601468086243),
 ('mandolin', 0.8618844747543335),
 ('classical', 0.8593218922615051),
 ('uke', 0.852906346321106),
 ('violin', 0.8483079075813293),
 ('alvarez', 0.843752920627594),
 ('banjo', 0.8360181450843811),
 ('strat', 0.8355288505554199)]

In [116]:
from_model = []


for word in chosen:
    try:
        similar = model.wv.most_similar(word, topn=2)  # не хочется добавлять много
        for sim in similar:
            if sim[0] not in chosen and sc(sim[0])[0].pos_ == 'NOUN' and len(sim[0]) > 2:
                from_model.append(sc(sim[0])[0].lemma_)
    except:  # на случай, если ключа нет в модели
        pass

In [117]:
len(from_model)

71

In [118]:
from_model

['water',
 'glue',
 'gator',
 'logo',
 'pedal',
 'wave',
 'planet',
 'condenser',
 'strap',
 'rehearsal',
 'interface',
 'band',
 'preamp',
 'track',
 'track',
 'home',
 'class',
 'kill',
 'input',
 'baritone',
 'padding',
 'pocket',
 'file',
 'store',
 'tube',
 'amp',
 'bag',
 'video',
 'house',
 'moment',
 'maintenance',
 'desk',
 'butter',
 'pad',
 'hole',
 'swivel',
 'repair',
 'business',
 'tune',
 'construction',
 'rubbery',
 'hofner',
 'fingertip',
 'supply',
 'gibson',
 'tune',
 'kit',
 'device',
 'interface',
 'operation',
 'bottom',
 'front',
 'tone',
 'pick',
 'extension',
 'month',
 'week',
 'price',
 'cost',
 'holder',
 'pivot',
 'primary',
 'keyboard',
 'mandolin',
 'pound',
 'car',
 'gibson',
 'interface',
 'compression',
 'chorus',
 'cable']

In [119]:
complete = chosen | set(from_model)

In [120]:
len(complete)

159

In [121]:
complete

{'6pc',
 'abs',
 'accessory',
 'allpart',
 'amp',
 'art',
 'audio',
 'bag',
 'band',
 'baritone',
 'bass',
 'behringer',
 'body',
 'boss',
 'bottom',
 'business',
 'butter',
 'cable',
 'car',
 'case',
 'channel',
 'chorus',
 'clamp',
 'clarinet',
 'class',
 'clip',
 'color',
 'compression',
 'condenser',
 'construction',
 'cost',
 'custom',
 'desk',
 'device',
 'dozen',
 'drum',
 'drumstick',
 'dvd',
 'ear',
 'effect',
 'end',
 'epiphone',
 'extension',
 'fender',
 'file',
 'fingertip',
 'flute',
 'foot',
 'front',
 'gator',
 'gibson',
 'glue',
 'gold',
 'guitar',
 'head',
 'headset',
 'hofner',
 'holder',
 'hole',
 'home',
 'house',
 'input',
 'instrument',
 'interface',
 'key',
 'keyboard',
 'kill',
 'kit',
 'leather',
 'led',
 'length',
 'light',
 'lighting',
 'line',
 'logo',
 'machine',
 'maintenance',
 'mandolin',
 'medium',
 'microphone',
 'mixer',
 'model',
 'moment',
 'month',
 'mp3',
 'music',
 'nickel',
 'note',
 'nut',
 'operation',
 'ovation',
 'pack',
 'package',
 'pad',


Очевидный минус - слишком много характеристик товаров (типа color, size, year). Возможно, метод №2 был бы лучше метода №3: мы бы нашли синонимы только нужных нам существительных, а не всех, встречающихся в заголовках товаров

3. (1 балл) Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед)

In [135]:
my_sw = {'the', 'a', 'is', 'this', 'that', 'or', 'to', 'of', 'my', 'was', 'has', 'with', 'like', 'as', 'at', 'for'}
pairs = set()

for sent in sentences:
    for i, word in enumerate(sent):
        if word in complete:
            if i > 0 and sent[i - 1] not in my_sw:
                pairs.add(sent[i - 1] + ' ' + word)
            if i != len(sent) - 1 and sent[i + 1] not in my_sw:
                pairs.add(word + ' ' + sent[i + 1])

In [136]:
pairs

{'and clarinet',
 'bottom will',
 'rubber piece',
 'side wall',
 'house feel',
 'twin tube',
 'solid desk',
 'pocket inside',
 'single light',
 'model if',
 'volume control.it',
 '(which swivel',
 'pedal strikes',
 'endless supply',
 'up tuning',
 'guitars volume',
 'stage headphones',
 'swivel joint',
 'clip because',
 'heavy head',
 'style mandolin',
 'goth studio',
 'perfect strap',
 'year old,',
 'plastic package',
 'winder/cutter/bridge pin',
 'glue rectified',
 'doing studio',
 'poor volume',
 '1970 gibson',
 'model features',
 'supply, input',
 'accurate tuning',
 'audio racks',
 'can sound',
 'through amp',
 'boss os2',
 'good price',
 '335-style case',
 'guitar type',
 'pedal though.',
 'light breakup',
 'princeton chorus',
 'fender amp',
 'it custom',
 'some tube',
 'gibson gospel',
 'pedal selection,',
 'piece into',
 'all planet',
 'octagon hole',
 'better tune',
 'tome machine',
 'old pad',
 'chorus ...good',
 'effect here',
 'included usb',
 'winder, string',
 '3 string',

In [139]:
trigrams = set()

for sent in sentences:
    for i, word in enumerate(sent):
        if word in complete:
            if i > 1 and sent[i - 1] not in my_sw and sent[i - 2] not in my_sw:
                trigrams.add(" ".join(sent[i - 2:i + 1]))
            if i > 0 and i != len(sent) - 1 and sent[i + 1] not in my_sw and sent[i - 1] not in my_sw:
                trigrams.add(" ".join(sent[i - 1:i + 2]))
            if i < len(sent) - 2 and sent[i + 1] not in my_sw and sent[i + 2] not in my_sw:
                trigrams.add(" ".join(sent[i:i + 3]))

In [140]:
trigrams

{'surround sound option.',
 'forward in sound',
 'boss pedal tuners).',
 'fold up stand',
 'fender amps, and',
 'guitar polish (already',
 'and condenser microphones.here',
 'hours in guitar',
 'guitar caseto store',
 "further, it's class",
 'plus drum and',
 'tone they are',
 'chorus which can',
 'medium sized venue.',
 'usb cable (mini',
 'acoustic guitar quick',
 "sunburst color (it's",
 'guitar and their',
 '&#34;tinny&#34;.i should note',
 'tube amp should',
 'set screw allows',
 'color differences. then',
 '"these strings sound',
 'ball type mic',
 'effective akai keyboard',
 'bag pocket and',
 'power level. i',
 'note and have',
 '&#34; and set',
 'guitar strap button.',
 'no guitar effects',
 '100 watt amp.',
 'guitars and amp',
 'tone wicker muff',
 'pedal can overdrive',
 'studio deluxe plustopfender',
 'price for. it',
 'head stock fits',
 'general guitar work,',
 "guitar's. two fender",
 'these strings...they sound',
 'style head and',
 'head you can',
 'your guitar without

4. (3 балла) Ранжируйте n-граммы с помощью 3 коллокационных метрик (t-score, PMI и т.д.). Не забудьте про частотный фильтр / сглаживание.
Выберите лучший результат (какая метрика  ранжирует выше коллокации, подходящие для отчёта).

In [142]:
import nltk
from nltk.collocations import *


bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [222]:
# В документации nltk метрики считают через CollocationFinder, который принимает лист с токенами. 
# Возможно, есть способ сразу посчитать метрики для списка n-грамм. 
# Пока, в качестве костыля, просто соединим наши n-граммы в один список

joined_pairs = []
for p in pairs:
    joined_pairs.extend(p.split())
    
joined_trigrams = []
for t in trigrams:
    joined_trigrams.extend(t.split())

In [223]:
bi_finder = BigramCollocationFinder.from_words(joined_pairs)
tri_finder = TrigramCollocationFinder.from_words(joined_trigrams)
bi_finder.apply_freq_filter(5)
tri_finder.apply_freq_filter(5)

In [224]:
# PMI

bi_finder.nbest(bigram_measures.pmi, 10)

[('from', 'strap'),
 ('so', 'string'),
 ('up', 'sound'),
 ('fender', 'bag'),
 ('in', 'bass'),
 ('style', 'bass'),
 ('microphone', 'end'),
 ('and', 'stand'),
 ('sound', 'supply'),
 ('performance', 'guitar')]

In [225]:
tri_finder.nbest(trigram_measures.pmi, 10)

[('guitar', 'planet', 'waves'),
 ('and', 'planet', 'waves'),
 ('planet', 'waves', 'guitar'),
 ('guitar', 'strap', 'locks'),
 ('power', 'supply', 'and'),
 ('and', 'clip', 'on'),
 ('and', 'good', 'price'),
 ('and', 'electric', 'guitar'),
 ('acoustic', 'guitar', 'sound'),
 ('electric', 'guitar', 'sound')]

In [226]:
# student_t

bi_finder.nbest(bigram_measures.student_t, 10)

[('performance', 'guitar'),
 ('fender', 'bag'),
 ('from', 'strap'),
 ('so', 'string'),
 ('up', 'sound'),
 ('style', 'bass'),
 ('in', 'bass'),
 ('and', 'stand'),
 ('sound', 'supply'),
 ('microphone', 'end')]

In [227]:
tri_finder.nbest(trigram_measures.student_t, 10)

[('and', 'electric', 'guitar'),
 ('and', 'tone', 'and'),
 ('guitar', 'planet', 'waves'),
 ('and', 'planet', 'waves'),
 ('guitar', 'strap', 'locks'),
 ('acoustic', 'guitar', 'sound'),
 ('on', 'your', 'guitar'),
 ('and', 'your', 'guitar'),
 ('planet', 'waves', 'guitar'),
 ('power', 'supply', 'and')]

In [228]:
# jaccard

bi_finder.nbest(bigram_measures.jaccard, 10)

[('style', 'bass'),
 ('and', 'stand'),
 ('microphone', 'end'),
 ('sound', 'guitar'),
 ('set', 'volume'),
 ('fender', 'bag'),
 ('from', 'strap'),
 ('sound', 'sound'),
 ('tube', 'music'),
 ('in', 'bass')]

In [229]:
tri_finder.nbest(trigram_measures.jaccard, 10)

[('guitar', 'planet', 'waves'),
 ('and', 'planet', 'waves'),
 ('guitar', 'strap', 'locks'),
 ('planet', 'waves', 'guitar'),
 ('on', 'your', 'guitar'),
 ('and', 'tone', 'and'),
 ('power', 'supply', 'and'),
 ('and', 'electric', 'guitar'),
 ('and', 'clip', 'on'),
 ('in', 'your', 'guitar')]

Все три варианат получились так себе. Выглядит так, будто их нужно отфильтровать по частям речи. Я попробовала извлечь n-граммы просто из корпуса (не из списка n-грамм) - у jaccard получались очень хорошие варианты.

5. (1 балл) Сгруппируйте полученные коллокации по NE, выведите примеры для 5 товаров.